In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pickle
import pandas as pd
from algorithm import IMUAlgorithm
import transforms3d as t3d

In [ ]:
with open('./measurement.pkl', 'rb') as f:
    measurement = pickle.load(f)
pos = measurement['pos']
timestamp = measurement['timestamp']
gt = []

for _ in range(6):
    gt += list(np.linspace([0,0,0],[0,0.16,0], 250)) + list(np.linspace([0,0.16,0],[0.16,0.16,0], 250)) +list(np.linspace([0.16,0.16,0],[0.16,0,0], 250)) +list( np.linspace([0.16,0,0],[0,0,0], 250))

gt = np.array(gt)
gt = gt[:len(pos)]

yaw = 10 * np.pi / 180
pose_mat = t3d.euler.euler2mat(0,0,yaw)
for i in range(len(gt)):

    gt[i,:] = pose_mat @ gt[i,:]

### Get GT

In [ ]:
start_idx = 100
end_idx =5000
gt= gt[start_idx:end_idx]
pos = pos[start_idx:end_idx]
timestamp = timestamp[:end_idx-start_idx]

fig = plt.figure(figsize=(16, 16))
ax = fig.add_subplot(111)
ax.scatter(gt[:, 0], gt[:, 1], s=2, c='b')
ax.scatter(pos[:, 0], pos[:, 1], s=2, c='r')
plt.gca().set_aspect('equal', adjustable='box')
ax.set_title('GT - POS')

In [ ]:
diff = np.sqrt(np.sum(((pos - gt)**2), axis=1))

In [ ]:
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111)
ax.scatter(timestamp, diff,s=2, c='b')
ax.set_xlabel('Time (s)')
ax.set_ylabel('Distance (m)')
ax.set_title('GT - POS')

In [ ]:
diff_filted = IMUAlgorithm.filter_middle(diff, windows_sz=400)

In [ ]:
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111)
ax.scatter(timestamp[:3000], diff_filted[:3000],s=2, c='b')
ax.set_xlabel('Time (s)')
ax.set_ylabel('Distance (m)')
ax.set_title('Difference between IMU tracking results and actual motion')